In [1]:
#load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random
from functions import closest_point

In [2]:
#load chrom sizes (from https://github.com/igvteam/igv/blob/master/genomes/sizes/hg38.chrom.sizes)
chrom_size_df = pd.read_csv('../data/hg38.chrom.sizes', sep='\t', 
                            header=None, names=['Chr','Length'])

#create lst of chr IDs
chromosomes = ['chr'+str(i) for i in range(1,23)] + ['chrX','chrY']
chrom_size_df = chrom_size_df[chrom_size_df.Chr.isin(chromosomes)]
genome_length = sum(chrom_size_df.Length) #3,088,269,832

In [ ]:
#load df with conserved genomic regions
conserved_DCIs = pd.read_csv('../output/conserved_DCIs_thr7.txt', sep='\t')
#load HML2 sequences
hml2_short = pd.read_csv('../output/hml2_df_short.txt', sep='\t')
#load DCIs closed to HML2
DCIs_hml2 = pd.read_csv('../output/DCIs_and_HML2_thr7.txt', sep='\t')
print("{} DCIs close to HML2".format(DCIs_hml2.shape[0]))

In [ ]:
DCIs_hml2.sort_values(by='Distance', ascending=True).reset_index(drop=True)

In [ ]:
#assuming we are stretching out the genome, 
#assign each chromosome its corresponding interval 
counter = 0 
chr_intervals = []

for chrom in chrom_size_df.Chr.unique():
    chrom_size = chrom_size_df.loc[chrom_size_df.Chr==chrom]['Length'].values[0]
    interval = (counter, counter+chrom_size)
    chr_intervals.append(interval)
    counter += chrom_size
    
chrom_size_df['Chr_Intervals'] = chr_intervals
chrom_size_df.head()

In [6]:
#initialize chrom dict to populate randomly scattered TADs
random_tads = {}
random_tads = random_tads.fromkeys(chrom_size_df.Chr.unique(),[])

In [ ]:
beginning = time.time()
num_simulations = 2000
simulated_results = []

for i in range(num_simulations):   
    
    #keep track of progress
    if i%500==0:
        print('Simulation #{}'.format(i))
    
    #pick 19 random genomic sites to place DCIs/TADs
    #19 because we found this number of DCIs close to HML2 sequences
    random_TADs = random.sample(range(1, genome_length), 19)

    #initialize dict to populate with random coordinates (by Chr)
    random_tads_adj = {}
    random_tads_adj = random_tads_adj.fromkeys(chrom_size_df.Chr.unique(),[])

    #populate dict created above
    for tad in random_TADs:
        #assign TAD to a chrom
        boolean_lst = [tad in range(start+1, end) for start,end in chr_intervals]
        chrom_loc = chrom_size_df.loc[boolean_lst]['Chr'].values[0] #chr ID
        #correct values so that they represent coordinates within a chromosome
        start_site = chrom_size_df.loc[chrom_size_df.Chr==chrom_loc]['Chr_Intervals'].values[0][0] #1st value in tuple
        #location in chr = location on linear genome - start coord of chr
        random_tads_adj[chrom_loc] = random_tads_adj[chrom_loc]+[tad-start_site] #add location to chr list

    ##create df from dict with random sites
    col1 = []
    col2 = []
    
    #iterate through chromosomes who got a random site
    for key in random_tads_adj:
        n_entries = len(random_tads_adj[key])
        if n_entries > 0:
            #create lists for df
            col1 = col1+([key]*n_entries)
            col2 = col2+random_tads_adj[key]
    random_TADs_df = pd.DataFrame(list(zip(col1, col2)), columns =['Chr', 'Start'])
    
    #create df template
    conserved_random = pd.DataFrame(columns=['Chr', 'Start', 'Coordinate', 'ClosestTAD'])

    for chrom in chromosomes:
        #skip if chrom is not present in randomly generated df or in conserved DCIs
        if chrom not in random_TADs_df.Chr.unique():
            continue
        elif chrom not in conserved_DCIs.Chr.unique():
            continue
            
        df_conserved = conserved_DCIs[conserved_DCIs.Chr==chrom].copy()
        df_random = random_TADs_df[random_TADs_df.Chr==chrom].copy()

        #get linear coordinates for each "TAD"
        df_random['Coordinate'] = [(x, y) for x,y in zip(df_random['Start'], pd.Series([0]).repeat(df_random.shape[0]))]
        df_conserved['Coordinate'] = [(x, y) for x,y in zip(df_conserved['Start'], pd.Series([0]).repeat(df_conserved.shape[0]))]
        df_random['ClosestTAD'] = [closest_point(x, list(df_conserved['Coordinate'])) for x in df_random['Coordinate']]
        
        #calculate how far matched TADs are from each other
        coord2 = [tup[0] for tup in df_random['ClosestTAD']]
        df_random['Distance'] = [a - b for a,b in zip(df_random['Start'], coord2)]
        df_random.sort_values(by='Start', ascending=True, inplace=True)
        
        #concat to template df
        conserved_random = pd.concat([conserved_random, df_random])

    conserved_random.reset_index(drop=True, inplace=True)
    conserved_random['AbsDistance'] = abs(conserved_random.Distance)
    
    #metrics to collect for each iteration
    #accumulated distance
    dist_sum = sum(conserved_random.AbsDistance)
    dist_mean = np.mean(conserved_random.AbsDistance)
    dist_median = np.median(conserved_random.AbsDistance)
    
    simulated_results.append([dist_sum,dist_mean,dist_median])
    
end = time.time()
print('Elapsed time: {} minutes'.format(round(end-beginning)/60,2))

In [8]:
#create df with results
all_dists = [lst[0] for lst in simulated_results]
all_means = [lst[1] for lst in simulated_results]
all_medians = [lst[2] for lst in simulated_results]
metrics_df = pd.DataFrame(list(zip(all_dists, all_means, all_medians)), 
                          columns=['Distances','Means','Medians'])

#define dfs with "true" observed distances
control_df1 = DCIs_hml2.copy()
#control_df2 has the "best" HML2 sequences (based on strongest normalized counts)
control_df2 = control_df1[(control_df1['KO_RANK']==1.0)|(control_df1['KO_RANK']==2.0)]

In [ ]:
# Plot permutation simulations (1) -- accumulated distances
plt.figure(figsize=(8,6))
density_plot = sns.kdeplot(metrics_df.Distances, shade=True)

#increase font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Accumulated Distances', fontsize=14)
plt.ylabel('Number of Simulations', fontsize=14)

# Add a line to show the actual difference observed in the data
density_plot.axvline(x=sum(control_df1.Distance), color='black', linestyle='--')
density_plot.axvline(x=sum(control_df2.Distance), color='crimson', linestyle='--')
labels = ['All HML2s', 'Highly Expressed\nHML2s', 'Simulated HML2s']
plt.legend(labels=labels, loc='upper right', fontsize='large')

#add p-value texts
simulations_less_than_observed = sum(metrics_df.Distances <= sum(control_df1.Distance))
p_value1 = simulations_less_than_observed / num_simulations
simulations_less_than_observed = sum(metrics_df.Distances <= sum(control_df2.Distance))
p_value2 = simulations_less_than_observed / num_simulations

plt.text(0.7, 0.5, 'P-value: {}'.format(round(p_value1,3)), 
         fontsize=14, color='black',
         transform=plt.gcf().transFigure,)
plt.text(0.7, 0.45, 'P-value: {}'.format(round(p_value2,3)), 
         fontsize=14, color='crimson',
         transform=plt.gcf().transFigure,)
# plt.savefig('../output/figures/permutation_test1_acc.png',
#             dpi=300, bbox_inches='tight')
# plt.show()

print('P-value for observed HML2 distances: {}'.format(p_value1))
print('P-value for best HML2 sequences: {}'.format(p_value2))

In [ ]:
# Plot permutation simulations (2) -- mean distances
plt.figure(figsize=(8,6))
density_plot = sns.kdeplot(metrics_df.Means, shade=True)

#increase font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Mean Distances', fontsize=14)
plt.ylabel('Number of Simulations', fontsize=14)

# Add a line to show the actual difference observed in the data
density_plot.axvline(x=np.mean(control_df1.Distance), color='black', linestyle='--')
density_plot.axvline(x=np.mean(control_df2.Distance), color='crimson', linestyle='--')
plt.legend(labels=labels, loc='upper right', fontsize='large')

#add p-value texts
simulations_less_than_observed = sum(metrics_df.Means <= np.mean(control_df1.Distance))
p_value1 = simulations_less_than_observed / 2000
simulations_less_than_observed = sum(metrics_df.Means <= np.mean(control_df2.Distance))
p_value2 = simulations_less_than_observed / 2000

plt.text(0.7, 0.5, 'P-value: {}'.format(round(p_value1,3)), 
         fontsize=14, color='black',
         transform=plt.gcf().transFigure,)
plt.text(0.7, 0.45, 'P-value: {}'.format(round(p_value2,3)), 
         fontsize=14, color='crimson',
         transform=plt.gcf().transFigure,)
# plt.savefig('../output/figures/permutation_test2_mean.png',
#             dpi=300, bbox_inches='tight')
plt.show()

print('P-value for observed HML2 distances: {}'.format(p_value1))
print('P-value for best HML2 sequences: {}'.format(p_value2))

In [ ]:
# Plot permutation simulations (3) -- median distances
plt.figure(figsize=(8,6))
density_plot = sns.kdeplot(metrics_df.Medians, shade=True)

#increase font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Median Distances', fontsize=14)
plt.ylabel('Number of Simulations', fontsize=14)

# Add a line to show the actual difference observed in the data
density_plot.axvline(x=np.median(control_df1.Distance), color='black', linestyle='--')
density_plot.axvline(x=np.median(control_df2.Distance), color='crimson', linestyle='--')
plt.legend(labels=labels, loc='upper right', fontsize='large')

#add p-value texts
simulations_less_than_observed = sum(metrics_df.Medians <= np.median(control_df1.Distance))
p_value1 = simulations_less_than_observed / 2000
simulations_less_than_observed = sum(metrics_df.Medians <= np.median(control_df2.Distance))
p_value2 = simulations_less_than_observed / 2000

plt.text(0.7, 0.5, 'P-value: {}'.format(round(p_value1,3)), 
         fontsize=14, color='black',
         transform=plt.gcf().transFigure,)
plt.text(0.7, 0.45, 'P-value: {}'.format(round(p_value2,3)), 
         fontsize=14, color='crimson',
         transform=plt.gcf().transFigure,)
plt.savefig('../output/figures/permutation_test3_median.png',
            dpi=300, bbox_inches='tight')
plt.show()

print('P-value for observed HML2 distances: {}'.format(p_value1))
print('P-value for best HML2 sequences: {}'.format(p_value2))

In [ ]:
# Compare densities
plt.figure(figsize=(8,6))
density_plot = sns.kdeplot(conserved_random.Distance, shade=True)
density_plot2 = sns.kdeplot(control_df1.Distance, shade=True)
density_plot3 = sns.kdeplot(control_df2.Distance, shade=True)
labels = ['Simulated HML2s', 'All HML2s', 'Highly Expressed\nHML2s']
plt.legend(labels=labels, loc='upper left', fontsize='large')

density_plot.set(xlabel='Distances', ylabel='Instances')
plt.show()